In [2]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
import openai
import pandas as pd


In [3]:
openai.api_key = "sk-"

In [4]:
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000, openai_api_key=openai.api_key),
    tool=PythonREPLTool(),
    verbose=True
)

## Download all designers names from the main pages

In [5]:
agent_executor.run("scrappe all href names of fall-2023-ready-to-wear from the web page https://www.vogue.com/fashion-shows")



> Entering new AgentExecutor chain...
 I need to use a web scraper to get the href names
Action: Python REPL
Action Input: 
import requests
from bs4 import BeautifulSoup

url = 'https://www.vogue.com/fashion-shows'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

links = soup.find_all('a', href=True)

Observation: 
Thought: I need to filter out the href names for fall-2023-ready-to-wear
Action: Python REPL
Action Input: 
fall_2023_links = [link['href'] for link in links if 'fall-2023-ready-to-wear' in link['href']]

Observation: 
Thought: I now know the final answer
Final Answer: fall_2023_links

> Finished chain.


'fall_2023_links'

In [6]:
from bs4 import BeautifulSoup
import requests
page = requests.get('https://www.vogue.com/fashion-shows')
soup = BeautifulSoup(page.content, 'html.parser')
#hrefs = [a['href'] for a in soup.find_all('a', href=True)]
links = soup.find_all('a', href=True)

fall_2023_links = []
for link in links:
    if 'fall-2023-ready-to-wear' in link['href']:
        fall_2023_links.append(link['href'])


In [7]:
len(fall_2023_links)

376

## Download all text about fashion show and images from designers pages

In [8]:
agent_executor.run("scrappe all the text and images from https://www.vogue.com/fashion-shows/fall-2023-ready-to-wear/y-project")



> Entering new AgentExecutor chain...
 I need to use a web scraper to get the text and images from the website
Action: Python REPL
Action Input: 
import requests
from bs4 import BeautifulSoup

url = 'https://www.vogue.com/fashion-shows/fall-2023-ready-to-wear/y-project'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

Observation: 
Thought: I need to get the text and images from the page
Action: Python REPL
Action Input: 
text = soup.find_all('p')
images = soup.find_all('img')

Observation: 
Thought: I now have the text and images from the page
Final Answer: The text and images from the page can be accessed using the variables 'text' and 'images'.

> Finished chain.


"The text and images from the page can be accessed using the variables 'text' and 'images'."

In [9]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.vogue.com/fashion-shows/fall-2023-ready-to-wear/y-project'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
text = soup.find_all('p')
images = soup.find_all('img')

In [10]:
agent_executor.run("scrappe all the images src link from https://www.vogue.com/fashion-shows/fall-2023-ready-to-wear/y-project")



> Entering new AgentExecutor chain...
 I need to use a web scraper to get the image src links
Action: Python REPL
Action Input: 
import requests
from bs4 import BeautifulSoup

url = 'https://www.vogue.com/fashion-shows/fall-2023-ready-to-wear/y-project'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

images = soup.find_all('img')

Observation: 
Thought: I need to get the src links from the images
Action: Python REPL
Action Input: 
image_src_links = []
for image in images:
    image_src_links.append(image['src'])

Observation: 
Thought: I now know the final answer
Final Answer: The image src links are: [image_src_links]

> Finished chain.


'The image src links are: [image_src_links]'

In [11]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.vogue.com/fashion-shows/fall-2023-ready-to-wear/y-project'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

images = soup.find_all('img')
image_src_links = []
for image in images:
    image_src_links.append(image['src'])

In [12]:
df_text = pd.DataFrame()
df_image = pd.DataFrame()
text_n = 0
image_n = 0
for designer_show in fall_2023_links:
    designer_name = designer_show.split("/")[-1]
    print(f"Downloading data of {designer_name}")
    df_text.loc[text_n,"designer"] = designer_name
    url = f'https://www.vogue.com{designer_show}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    if soup.find('div', class_='body__inner-container'):
        texts = soup.find('div', class_='body__inner-container').get_text()
        df_text.loc[text_n,"text_runway"] = texts
        text_n += 1
    images = soup.find_all('img')
    image_src_links = []
    for image in images:
        if "runway" in image['src'] and "https" in image['src']:
            image_src_links.append(image['src'])
            df_image.loc[image_n,"designer"] = designer_name
            df_image.loc[image_n,"image_link"] = image['src']
            image_n += 1

In [13]:
df_text.head()

,designer,text_runway
0,a-company,“I feel protected by history.” So said A--Comp...
1,a-l-c,Real clothes won at New York fashion week this...
2,a-p-c-,“The kids are all right” was Jean Touitou’s su...
3,a-w-a-k-e-,Natalia Alaverdian arrived at Paris Fashion We...
4,acne-studios,With crystal strands dripping from twisted tre...


In [14]:
print(f"Designers from vogue runway fashion shows: {len(df_image)}")

Designers from vogue runway fashion shows: 2254


In [15]:
df_image.head()

,designer,image_link
0,a-p-c-,https://assets.vogue.com/photos/6404d2f8068d12...
1,a-p-c-,https://assets.vogue.com/photos/6404d2bbc12f0b...
2,a-p-c-,https://assets.vogue.com/photos/6404d2bc194566...
3,a-p-c-,https://assets.vogue.com/photos/6404da4544574b...
4,a-p-c-,https://assets.vogue.com/photos/6404d2be7b0104...


In [16]:
print(f"Images obtained from vogue runway fashion shows: {len(df_image)}")

Images obtained from vogue runway fashion shows: 2254


In [22]:
df_image.to_csv("image_vougue_runway.csv", index=False)
df_text.to_csv("text_vougue_runway.csv", index=False)

In [23]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ChatVectorDBChain
import pickle

In [24]:
loader = CSVLoader("text_vougue_runway.csv")
raw_documents = loader.load()

In [32]:
import sys
sys.setrecursionlimit(10000) 

In [35]:
len(raw_documents)

354

In [36]:

text_splitter = RecursiveCharacterTextSplitter(separators=" ")
documents = text_splitter.split_documents(raw_documents)
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
vectorstore = FAISS.from_documents(documents, embeddings)
with open("vogue_shows.pkl", "wb") as f:
    pickle.dump(vectorstore, f)
    

In [37]:
def get_chain(vectorstore):
    llm = OpenAI(temperature=0, openai_api_key=openai.api_key, model_name="gpt-3.5-turbo")
    qa_chain = ChatVectorDBChain.from_llm(
        llm,
        vectorstore,
        top_k_docs_for_context=3
    )
    return qa_chain

In [38]:
qa_chain = get_chain(vectorstore)

/home/lpastur/miniconda3/envs/web3MBA/lib/python3.11/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/lpastur/miniconda3/envs/web3MBA/lib/python3.11/site-packages/langchain/llms/openai.py:623: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/lpastur/miniconda3/envs/web3MBA/lib/python3.11/site-packages/langchain/chains/conversational_retrieval/base.py:191: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warnings.warn(


In [39]:
def ask_question(question):
    chat_history = []
    result = qa_chain({"question": question, "chat_history": chat_history})
    print(result['answer'])

In [41]:
question = "How is the Bottega Veneta collection?"
ask_question(question)

The Bottega Veneta collection is described as a knock-out and a rare feat that simultaneously ups the fashion ante while also delivering real-life relatability. The collection is diverse and adheres to no developing trends because together they were all different. The designer, Matthieu Blazy, started with the street and the idea of the strange encounter—people that you meet in the street and they really amaze you. The collection includes breakthrough leather tank tops and leather jeans, layered dresses with sweet flower embroideries, deconstructed 1950s screen star dresses, and an exceptional LBD with a swooping neckline and a front slit not quite high enough to reveal the top of over-the-knee intrecciato boots. The materials used are light, unconstrained fabrics, and the silhouettes sometimes went to extremes.


In [42]:
question = "Describe 3 items from the Bottega Veneta collection?"
ask_question(question)

A breakthrough leather tank top, leather jeans, and a fringed coat woven in one piece.


In [43]:
question = "Describe all items from the Bottega Veneta collection?"
ask_question(question)

The Bottega Veneta collection featured 81 looks that were all different and did not adhere to any developing trends. The collection started with a sheer dressing gown and house shoes, and included breakthrough leather tank tops and leather jeans. There were layered dresses with sweet flower embroideries, deconstructed 1950s screen star dresses, and an exceptional LBD with a swooping neckline and a front slit not quite high enough to reveal the top of over-the-knee intrecciato boots. The collection also featured cleanly tailored double-layer coats and jackets for women and men, as well as bulbous jelly pumps. Materials-wise, the designer used light, unconstrained fabrics, and shaved leather to make it more weightless. The collection also included a fringed coat that was woven in one piece, and rolled waistband skirts that were meant to conjure the fishtail bottom half of mermaids. The designer's inclination this season was to start with the street, and “the idea of the strange encounter

In [44]:
question = "Describe three items from the Bottega Veneta collection as an online product description for a ecommerce"
ask_question(question)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


1. Sheer dressing gown and house shoes: This luxurious dressing gown is perfect for lounging at home, with its sheer fabric and comfortable fit. The matching house shoes are made from knitted leather, adding a touch of sophistication to your at-home look.

2. Leather tank top and jeans: Make a statement in this edgy leather tank top and jeans set. The tank top features a breakthrough design, while the jeans are made from weightless shaved leather for a comfortable fit.

3. LBD with intrecciato boots: This little black dress is anything but basic, with a swooping neckline and front slit that shows off your over-the-knee intrecciato boots. The perfect outfit for a night out on the town.
